In [1]:
import pandas as pd
from openfisca_ceq.tools.survey_scenario import build_ceq_survey_scenario
from openfisca_ceq.tools.indirect_taxation.tax_benefit_system_indirect_taxation_completion import indirect_tax_by_country
from openfisca_ceq.tools.data import year_by_country
from openfisca_ceq.tools.data_ceq_correspondence import (
    ceq_input_by_harmonized_variable,
    ceq_intermediate_by_harmonized_variable,
    non_ceq_input_by_harmonized_variable,
    )

# Build country scenario

## Country selection

In [2]:
country = "cote_d_ivoire"

In [3]:
# Parameters
country = "cote_d_ivoire"


In [4]:
year = year_by_country[country]
survey_scenario = build_ceq_survey_scenario(legislation_country = country, year = year)

/home/benjello/.virtualenvs/openfisca-ceq/local/lib/python3.7/site-packages/pandas/io/stata.py:1409: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  warnings.warn(msg, UnicodeWarning)


# Aggegates check

## Income variables

### Disagregated income variables

In [5]:
ceq_by_harmonized_variable = dict()
ceq_by_harmonized_variable.update(ceq_input_by_harmonized_variable)
ceq_by_harmonized_variable.update(ceq_intermediate_by_harmonized_variable)
ceq_by_harmonized_variable.update(non_ceq_input_by_harmonized_variable)
ceq_by_harmonized_variable['salaire_brut'] = 'salaire_brut'
ceq_by_harmonized_variable['salaire_super_brut'] = 'salaire_super_brut'
ceq_by_harmonized_variable

{'rev_i_autoconsommation': 'autoconsumption',
 'rev_i_autres': 'other_income',
 'rev_i_autres_transferts': 'gifts_sales_durables',
 'rev_i_loyers_imputes': 'imputed_rent',
 'rev_i_transferts_publics': 'direct_transfers',
 'rev_i_agricoles': 'revenu_agricole',
 'rev_i_autres_revenus_capital': 'autres_revenus_du_capital',
 'rev_i_independants_Ntaxe': 'revenu_informel_non_salarie',
 'rev_i_independants_taxe': 'revenu_non_salarie',
 'rev_i_independants': 'revenu_non_salarie_total',
 'rev_i_locatifs': 'revenu_locatif',
 'rev_i_pensions': 'pension_retraite',
 'rev_i_salaires_formels': 'salaire',
 'rev_i_salaires_informels': 'revenu_informel_salarie',
 'salaire_brut': 'salaire_brut',
 'salaire_super_brut': 'salaire_super_brut'}

In [6]:
data = [
    (harmonized_variable, openfisca_variable, survey_scenario.compute_aggregate(openfisca_variable, period = year) / 1e9) 
    for harmonized_variable, openfisca_variable in ceq_by_harmonized_variable.items()
    ]

In [7]:
revenus = pd.DataFrame(data, columns = ["harmonized", "openfisca", "aggregate"])
revenus

,harmonized,openfisca,aggregate
0,rev_i_autoconsommation,autoconsumption,1288.430155
1,rev_i_autres,other_income,579.719635
2,rev_i_autres_transferts,gifts_sales_durables,432.967997
3,rev_i_loyers_imputes,imputed_rent,430.184956
4,rev_i_transferts_publics,direct_transfers,8.438274
5,rev_i_agricoles,revenu_agricole,1426.492481
6,rev_i_autres_revenus_capital,autres_revenus_du_capital,74.726052
7,rev_i_independants_Ntaxe,revenu_informel_non_salarie,1345.538351
8,rev_i_independants_taxe,revenu_non_salarie,244.463898
9,rev_i_independants,revenu_non_salarie_total,1590.002249


### Agregated income variables

#### CEQ agregated income variables

In [8]:
ceq_variables = ["market_income", "market_income_plus_pensions", "pensions", "contributions_pensions", "net_market_income", "direct_taxes", "other_contributions", "contributions_health", ]

In [9]:
ceq_aggregates = pd.DataFrame(
    index = ceq_variables,
    columns = ['aggregate'],
    data = [
        survey_scenario.compute_aggregate(ceq_variable, period = year) / 1e9 
        for ceq_variable in ceq_variables
        ]
    )
ceq_aggregates

,aggregate
market_income,9971.900835
market_income_plus_pensions,10124.485788
pensions,354.079328
contributions_pensions,201.494369
net_market_income,10077.457214
direct_taxes,47.028573
other_contributions,50.777320
contributions_health,0.000000


#### Direct check

In [10]:
income_components_summed_in_total = list(ceq_by_harmonized_variable.values()) 
income_components_summed_in_total.remove("revenu_non_salarie_total")
income_components_summed_in_total.remove("direct_transfers")
income_components_summed_in_total.remove("salaire")
income_components_summed_in_total.remove("salaire_brut")
total_income = sum(
    survey_scenario.compute_aggregate(income_component, period = survey_scenario.year) 
    for income_component in income_components_summed_in_total
    ) 
total_income = (
    total_income 
    - survey_scenario.compute_aggregate("contributions_pensions", period = survey_scenario.year)
    )/ 1e9


In [11]:
from numpy.testing import assert_almost_equal
assert_almost_equal(ceq_aggregates.loc['market_income_plus_pensions'].values, total_income, decimal = 5) 
# print('market_income_plus_pensions = {} vs {} = total_income'.format(
#         ceq_aggregates.loc['market_income_plus_pensions'].values,
#         total_income
#         )
#     )


## Education variables

### Counts (in millions)

In [12]:
print((survey_scenario.compute_pivot_table(columns = ['eleve_enseignement_niveau'], index = ['eleve_enseignement_public'], aggfunc = 'count', period = survey_scenario.year) / 1e6).round(2))

eleve_enseignement_niveau   -1.0   0.0   1.0   2.0   3.0
eleve_enseignement_public                               
0.0                          NaN  0.06  0.54  0.55  0.07
1.0                        18.12  0.08  2.66  0.89  0.10


In [13]:
variables = [
    'pre_school_person',
    'pre_school',
    'primary_education_person',
    'primary_education',
    'secondary_education_person',
    'secondary_education',
    'tertiary_education_person',
    'tertiary_education',
    'education_net_transfers',
    ]
print("In kind education tranfers")
for variable in variables:
    print(
        "{variable}: {aggregate} billions FCFA".format(
            variable = variable,
            aggregate = int(round(survey_scenario.compute_aggregate(variable, period = survey_scenario.year) / 1e9))
            )
        )

In kind education tranfers
pre_school_person: 21 billions FCFA
pre_school: 21 billions FCFA
primary_education_person: 341 billions FCFA
primary_education: 341 billions FCFA
secondary_education_person: 309 billions FCFA
secondary_education: 309 billions FCFA
tertiary_education_person: 193 billions FCFA
tertiary_education: 193 billions FCFA
education_net_transfers: 865 billions FCFA


## Tax variables

### Indirect tax variables

In [14]:
indirect_tax_variables = [
    variable 
    for tax in indirect_tax_by_country[country]
    for variable in survey_scenario.tax_benefit_system.variables.keys() 
    if tax in variable 
    ]
indirect_tax_variables

['depenses_ht_tva_exonere',
 'depenses_ht_tva_taux_normal',
 'depenses_ht_tva_taux_reduit',
 'tva_taux_normal',
 'tva_taux_reduit',
 'tva']

In [15]:
 pd.DataFrame(
    index = indirect_tax_variables,
    columns = ['aggregate'],
    data = [
        survey_scenario.compute_aggregate(variable, period = year) / 1e9 
        for variable in indirect_tax_variables
        ]
    )

,aggregate
depenses_ht_tva_exonere,5857.616332
depenses_ht_tva_taux_normal,3977.434894
depenses_ht_tva_taux_reduit,403.919160
tva_taux_normal,715.938310
tva_taux_reduit,36.352726
tva,752.291035


In [16]:
survey_scenario.compute_aggregate('consumption', period = year) / 1e9

10991.261126886855

In [17]:
survey_scenario.tax_benefit_system.variables.keys()

dict_keys(['household_weight', 'person_weight', 'age', 'formel_informel', 'marie', 'nombre_de_parts', 'nombre_enfants_a_charge', 'occupation_principale_taxee', 'proprietaire_residence_principale', 'residence_principale_taxee', 'sexe', 'veuf_ve', 'aide_monetaire_autre_menage', 'aide_non_monetaire_autre_menage', 'autoconsommation', 'autre_revenus_individuels', 'bourses_transfers', 'interets_dividendes', 'pension_retraite', 'revenu_locatif', 'revenus_activites_secondaires', 'revenus_complementaires_elevage', 'revenus_complementaires_miel', 'salaire', 'salaire_brut', 'personal_income_tax', 'droits_de_douane', 'droits_de_douane_intermediaire', 'droits_de_douane_plein', 'droits_de_douane_reduit', 'impots_indirects', 'impot_general_revenu', 'impots_directs', 'salaire_net_a_payer', 'accidents_du_travail', 'cotisations_employeur', 'cotisations_salariales', 'famille', 'retraite_employeur', 'retraite_salarie', 'salaire_imposable', 'salaire_super_brut', 'sante_employeur', 'sante_salarie', 'contrib